In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import os

print("GRO620 - Problématique")
print("OpenCV version", cv.__version__)

%matplotlib inline

In [ ]:
# Load images

images_fn = os.listdir("photos_prob/")
images_fn.sort()
print("%i photo(s) à traiter" % (len(images_fn)))
if len(images_fn) == 0:
    print(
        "ERREUR! Vérifiez que vous avez bien un dossier photos_prob au même endroit que ce calepin."
    )

images: list[cv.typing.MatLike] = []

for f in images_fn:
    img_path = os.path.join("photos_prob/", f)
    img = cv.imread(img_path)
    assert img is not None
    img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    images.append(img)

plt.figure(0)
for index, image in enumerate(images):
    plt.subplot(3, 3, index + 1)
    plt.imshow(image)
    plt.title(f"Original {index + 1}")
    plt.axis("off")
plt.show()


In [ ]:
# 1. Parametres de la camera

FOCAL_LENGTH = 23.0e-3  # m
IMG_WIDTH = 640  # px
IMG_HEIGHT = 427  # px
SENSOR_WIDTH = 23.4e-3  # m
SENSOR_HEIGHT = 15.6e-3  # m

# Calcul des parametres intrinseques
fx = FOCAL_LENGTH * (IMG_WIDTH / SENSOR_WIDTH)  # px
fy = FOCAL_LENGTH * (IMG_HEIGHT / SENSOR_HEIGHT)  # px
cx = IMG_WIDTH / 2.0  # px
cy = IMG_HEIGHT / 2.0  # px
skew = 0.0  # px

K_tilde = np.array(
    [
        [fx, skew, cx, 0],
        [0, fy, cy, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1],
    ],
    dtype=np.float32,
)

print(f"Matrice intrinseque K:\n{K_tilde}")


In [ ]:
# 2. Matrice de transformation extrinseque w_T_c (repere {C} au repere {0})

w_T_c_tilde = np.array(
    [
        [1, 0, 0, 0.500],
        [0, -1, 0, 0.200],
        [0, 0, -1, 0.282],
        [0, 0, 0, 1],
    ],
    dtype=np.float32,
)

print(f"Matrice extrinseque w_T_c (Camera -> World):\n{w_T_c_tilde}\n")

# Matrice de projection complete

w_P_c_tilde = w_T_c_tilde @ np.linalg.inv(K_tilde)

print(f"Matrice de projection complete (Camera -> World):\n{w_P_c_tilde}\n")

# Calcul de Z_c
X_0 = 0  # m
Y_0 = 0  # m
Z_0 = 0  # m
z_conveyor = (
    w_T_c_tilde[2, 3] + w_T_c_tilde[2, 0] * X_0 + w_T_c_tilde[2, 1] * Y_0 + w_T_c_tilde[2, 3] * Z_0
)  # m

print(f"Z_c: {z_conveyor:.3f} m")


In [ ]:
from sklearn.cluster._hdbscan.hdbscan import HDBSCAN

processed = []

all_results = []

for index, image in enumerate(images):
    original = image.copy()
    blurred = cv.GaussianBlur(original, (7, 7), 3.0)
    gray = cv.cvtColor(blurred, cv.COLOR_RGB2GRAY)
    _, threshold = cv.threshold(gray, 180, 255, cv.THRESH_BINARY)
    kernel = np.ones((3, 3), dtype=np.uint8)
    opening = cv.morphologyEx(threshold, cv.MORPH_OPEN, kernel)
    dilated = cv.dilate(opening, kernel, iterations=1)
    processed.append(dilated)

    lines = cv.HoughLinesP(
        dilated, 2.0, np.pi / 180, 100, minLineLength=25, maxLineGap=5
    )

    # Cluster lines detected by Hough transform
    # NOTE: ideally, we want one line per screw, but this is rather difficult to obtain with clustering.
    midpoints = np.array(
        [
            [(line[0][0] + line[0][2]) / 2, (line[0][1] + line[0][3]) / 2]  # type: ignore
            for line in lines
        ]
    )
    clusterer = HDBSCAN(
        min_cluster_size=2
    )  # Large-ish `n` samples, medium `n` clusters, cluster by distance betweem nearest points
    labels = clusterer.fit_predict(midpoints)
    # Average lines for each cluster label
    clustered_lines = []
    for lbl in np.unique(labels):
        if lbl == -1:
            continue  # skip noise
        idxs = np.where(labels == lbl)[0]
        # Average the endpoints of all lines in this cluster
        avg_line = np.mean([lines[i][0] for i in idxs], axis=0)
        clustered_lines.append(avg_line.astype(int))
    lines = np.array(clustered_lines).reshape(-1, 1, 4)

    # print(f"Image {index + 1}: {len(lines)} lines detected")

    for line in lines:
        x1, y1, x2, y2 = line[0]
        theta = np.rad2deg(np.atan2(y2 - y1, x2 - x1))
        cv.line(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

    # Identify, classify, and compute pose
    contours, _ = cv.findContours(dilated, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    MIN_SCREW_AREA = 100  # px
    MAX_SCREW_AREA = 2000  # px
    HEIGHT_THRESHOLD = 100.0  # px

    # Extract caracterstics
    results = []
    screw_id: int = 0
    for contour in contours:
        area = cv.contourArea(contour)
        if area < MIN_SCREW_AREA or area > MAX_SCREW_AREA:
            continue

        rect = cv.minAreaRect(contour)
        (c_u, c_v), (width, height), theta = rect

        # Draw bounding box and id
        bbox = cv.boxPoints(rect)
        bbox = np.intp(bbox)  # Convert box points to integer type for drawing
        cv.drawContours(image, [bbox], 0, (0, 0, 255), 2)  # type: ignore
        cv.putText(
            image,
            str(screw_id),
            (int(c_u) - 5, int(c_v) - 5),
            cv.FONT_HERSHEY_SIMPLEX,
            1.0,
            (255, 0, 0),
            3,
            cv.LINE_AA,
        )

        # Standardise notation
        if width > height:
            width, height = height, width
            theta += 90
        theta = (theta + 90) % 180

        screw_type = "courte"
        if height > HEIGHT_THRESHOLD:
            screw_type = "longue"

        # # px -> m ({C} frame)
        z_screen = z_conveyor  # px?
        x_screen = c_u * z_screen  # px^2?
        y_screen = c_v * z_screen  # px^2?
        X_s_prime = np.array([x_screen, y_screen, z_screen, 1], dtype=np.float32)

        # # {C} -> {0}
        p_w = w_P_c_tilde @ X_s_prime
        x_0, y_0, z_0, _ = p_w

        results.append(
            {
                "id": screw_id,
                "type": screw_type,
                "X": x_0,
                "Y": y_0,
                "Z": z_0,
                "theta": theta,
            }
        )
        screw_id += 1

    all_results.append(results)

plt.figure(0)
for index, image in enumerate(processed):
    plt.subplot(3, 3, index + 1)
    plt.imshow(image, "gray")
    plt.title(f"Image {index + 1}")
    plt.axis("off")

plt.show()


In [ ]:
# Print results

import tabulate

for index, _ in enumerate(images):
    print(f"Image {index + 1}:\n")
    print(
        tabulate.tabulate(
            [
                [
                    screw["id"],
                    screw["type"],
                    screw["X"],
                    screw["Y"],
                    screw["Z"],
                    screw["theta"],
                ]
                for screw in all_results[index]
            ],
            headers=[
                "id",
                "Type",
                "X (m)",
                "Y (m)",
                "Z (m)",
                "theta (deg)",
            ],
            floatfmt=".3f",
        )
    )
    print()

plt.figure()
for index, image in enumerate(images):
    plt.subplot(3, 3, index + 1)
    plt.imshow(image)
    plt.title(f"Image {index + 1}")
    plt.axis("off")
plt.show()
